In [39]:
import pandas as pd
import numpy as np
#NGRIP then EDML
grip_data = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DJ-TALDICE.xls',
                        skiprows=0, names=['depth1', 'depth2'], usecols=[0,4], sheet_name=1, nrows=305)

In [43]:
grip_data['rel_unc'] = 20

grip_data['rel_unc'][grip_data['depth1']>1000] = 20

/var/folders/cd/1mh45k3j19g5g_kxkz_17xvc0000gn/T/ipykernel_11378/1092528484.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  grip_data['rel_unc'][grip_data['depth1']>1000] = 20
/var/folders/cd/1mh45k3j19g5g_kxkz_17xvc0000gn/T/ipykernel_11

In [44]:
grip_data

,depth1,depth2,rel_unc
0,1399.381,611.253,20
1,1419.080,623.500,20
2,1458.680,650.250,20
3,1470.640,659.520,20
4,1479.711,667.038,20
...,...,...,...
300,2925.540,2213.800,20
301,2929.680,2216.320,20
302,2936.190,2219.900,20
303,2946.480,2228.600,20


In [45]:
grip_data.to_csv('/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/NGRIP-EDML/iceice_synchrxo_horizons.txt', sep='\t', index=False, header=True)

In [5]:
import pandas as pd

read = pd.read_csv('/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/EDML-NGRIP/iceice_synchro_horizons.txt', delimiter='\t')

In [11]:
read

,depth1,depth2,rel_unc
0,1399.381,611.253,20
1,1419.080,623.500,20
2,1458.680,650.250,20
3,1470.640,659.520,20
4,1479.711,667.038,20
...,...,...,...
300,2925.540,2213.800,20
301,2929.680,2216.320,20
302,2936.190,2219.900,20
303,2946.480,2228.600,20


In [7]:
read = read.rename(columns={"rel_unc": "age_unc"})

print(read)

read.to_csv('/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/EDML-NGRIP/iceice_synchro_horizons2.txt', sep='\t', index=False)

       depth1    depth2  age_unc
0     611.253  1399.381       20
1     623.500  1419.080       20
2     650.250  1458.680       20
3     659.520  1470.640       20
4     667.038  1479.711       20
..        ...       ...      ...
300  2213.800  2925.540       20
301  2216.320  2929.680       20
302  2219.900  2936.190       20
303  2228.600  2946.480       20
304  2231.300  2949.880       20

[305 rows x 3 columns]
